In [27]:
#bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate 
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

In [2]:
#atribui os dados da uri para a variável dados
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [3]:
#define as features de x e y, SEED e cria treinos e testes de x e y.
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [4]:
#gera um dummy stratified para validação dos dados
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)

A acurácia do dummy stratified foi 50.96%


/usr/local/lib/python3.6/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [5]:
#cria um modelo de decision tree
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [6]:
#função para imprimir os resultados do intervalo e média com desvio padrão
def imprime_resultados(results):
  media = results ['test_score'].mean()
  desvio_padrao = results['test_score'].std()
  print("Accuracy intervalo [%.2f%%, %.2f%%]" % ((media - 2 *desvio_padrao) * 100, (media + 2 * desvio_padrao) * 100))
  print("Accuracy médio: %.2f%%" % (media * 100))

In [7]:
#aplica a validação cruzada no modelo
cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy intervalo [74.37%, 77.19%]
Accuracy médio: 75.78%


In [8]:
#aplica a validação cruzada com shuffle no modelo
cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy intervalo [73.34%, 78.22%]
Accuracy médio: 75.78%


In [9]:
#testando o modelo com o  stratified KFold
cv = StratifiedKFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy intervalo [73.79%, 77.77%]
Accuracy médio: 75.78%


In [10]:
len(dados)#verificando o tamanho do dataframe

10000

In [11]:
#cria a coluna 'modelo' com dados aleatórios gerados a partir do método random.randint() da biblioteca Numpy para testar valores desconhecidos pelo algoritmo a partir desses novos modelos
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1 #trata o valor mínimo para 0 em vez de -1
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,19
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,4


In [12]:
#testando um modelo com dados gerados aleatórios utilizado o group KFold
cv = GroupKFold(n_splits = 10,)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy intervalo [72.02%, 79.56%]
Accuracy médio: 75.79%


In [23]:
#testando um modelo SVC com os dados escalados utilizado o método StandardScaler
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi de %.2f%%" %acuracia)

A acurácia foi de 74.40%


In [28]:
#adicionando o pipeline do sklearn no modelo anterior
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

cv = GroupKFold(n_splits=10)
results = cross_validate(pipeline, x, y, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results)

Accuracy intervalo [73.25%, 80.14%]
Accuracy médio: 76.70%
